In [1]:
import time
import pyspark

from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Spark") \
    .config('spark.sql.repl.eagerEval.enabled', 'True') \
    .config("spark.driver.memory", "16g") \
    .config("spark.driver.maxResultSize", "16g") \
    .getOrCreate()

In [3]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")

from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation

dataset = [[Vectors.dense([1, 0, 0, -2])],
           [Vectors.dense([4, 5, 0, 3])],
           [Vectors.dense([6, 7, 0, 8])],
           [Vectors.dense([9, 0, 0, 1])]]

dataset = spark.createDataFrame(dataset, ['features'])

dataset

pearsonCorr = Correlation.corr(dataset, 'features', 'pearson').collect()[0][0]

pearsonCorr

print(str(pearsonCorr).replace('nan', 'NaN'))

spearmanCorr = Correlation.corr(dataset, 'features', method='spearman').collect()[0][0]

print(str(spearmanCorr).replace('nan', 'NaN'))

In [ ]:
#spark.conf.set("spark.executor.memory", '4g')
#spark.conf.set('spark.executor.cores', '3')
#spark.conf.set('spark.cores.max', '3')
#spark.conf.set("spark.driver.memory",'4g')

In [4]:
sc = spark.sparkContext

In [5]:
sc.version

'3.1.1'

In [6]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.sql.warehouse.dir', 'file:/home/jovyan/code/spark-warehouse'),
 ('spark.app.name', 'Spark'),
 ('spark.sql.repl.eagerEval.enabled', 'True'),
 ('spark.driver.maxResultSize', '16g'),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1620903746812'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.host', 'ee6b80857adf'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1620903747915'),
 ('spark.driver.port', '34487'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true')]

In [7]:
TcgaTargetGTEX_phenotype = spark.read.options(header=True, inferSchema=True, delimiter='\t').csv("../data/TcgaTargetGTEX_phenotype.txt.gz")

In [8]:
TcgaTargetGTEX_phenotype.createOrReplaceTempView("phenotype")

In [9]:
GTEx_breast = spark.sql("SELECT * FROM phenotype where _study = 'GTEX' and _primary_site = 'Breast'")

In [10]:
type(GTEx_breast)

pyspark.sql.dataframe.DataFrame

In [11]:
GTEx_breast.head()

Row(sample='GTEX-T6MN-0726-SM-32PML', detailed_category='Breast - Mammary Tissue', primary disease or tissue='Breast - Mammary Tissue', _primary_site='Breast', _sample_type='Normal Tissue', _gender='Male', _study='GTEX')

In [12]:
GTEx_breast_sample = GTEx_breast.select("sample").rdd.flatMap(lambda x: x).collect()

In [13]:
start = time.time()
TcgaTargetGtex_RSEM_Hugo_norm_count = spark.read.options(header=True, inferSchema=True, delimiter='\t').csv("../data/TcgaTargetGtex_RSEM_Hugo_norm_count.gz")
print("time:", time.time() - start)

time: 222.0337541103363


In [14]:
test = ['sample']
test.extend(GTEx_breast_sample)
GTEx_Breast_norm_count = TcgaTargetGtex_RSEM_Hugo_norm_count.select(test)

In [15]:
import gc
TcgaTargetGtex_RSEM_Hugo_norm_count.unpersist()
del TcgaTargetGtex_RSEM_Hugo_norm_count
gc.collect()

930

In [16]:
GTEx_Breast_norm_count.limit(5).toPandas()

,sample,GTEX-T6MN-0726-SM-32PML,GTEX-13NZB-2126-SM-5MR4Y,GTEX-ZU9S-1926-SM-5NQBP,GTEX-11DXW-0626-SM-5N9ER,GTEX-UPK5-2326-SM-3P5Z8,GTEX-VUSG-2226-SM-4KKZO,GTEX-ZVTK-0326-SM-51MRR,GTEX-14ABY-2126-SM-5QGQZ,GTEX-R55F-1726-SM-2TF4R,...,GTEX-12WSB-0626-SM-59HJO,GTEX-ZPIC-1126-SM-5BC7F,GTEX-132AR-0826-SM-5EGK6,GTEX-SE5C-2126-SM-4BRUJ,GTEX-ZV6S-1826-SM-5NQ8D,GTEX-145ME-1526-SM-5Q5F2,GTEX-WI4N-1426-SM-3LK7H,GTEX-13CF3-2126-SM-5IFJP,GTEX-S32W-2026-SM-4AD6E,GTEX-12WSK-2226-SM-5GCO5
0,CTD-2588J6.1,1.4482,0.0000,0.0000,0.0000,1.2686,0.0000,0.0000,0.0,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.7116,0.0000,0.0000
1,RP11-433M22.1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,CTD-2588J6.2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,RP3-415N12.1,2.8177,0.0000,1.4965,1.0942,1.9330,2.7404,0.0000,0.0,0.0000,...,1.6992,0.0000,0.0000,1.9692,0.0000,0.0000,2.4270,2.7416,2.2171,1.7514
4,RP11-16E12.2,3.5074,1.0154,1.9000,2.4699,2.7305,2.1406,2.1822,0.0,2.8314,...,3.6131,4.7796,2.9253,4.6043,3.7802,1.1093,4.8804,3.3365,5.2287,3.6819


In [17]:
start = time.time()
pandasDF = GTEx_Breast_norm_count.toPandas().set_index('sample').transpose()
print("time:", time.time() - start)

time: 80.27529382705688


In [18]:
GTEx_Breast_norm_count.unpersist()
del GTEx_Breast_norm_count
gc.collect()

68

In [19]:
gene = pandasDF.columns

In [20]:
gene

Index(['CTD-2588J6.1', 'RP11-433M22.1', 'CTD-2588J6.2', 'RP3-415N12.1',
       'RP11-16E12.2', 'RP11-433M22.2', 'PSMA2P3', 'RP11-181G12.4',
       'RP11-181G12.2', 'AC007272.3',
       ...
       'RP11-390F4.3', 'GNGT2', 'GNGT1', 'SELT', 'DIAPH2-AS1', 'SELV', 'NFIX',
       'SELP', 'AC002386.1', 'RP11-526P5.2'],
      dtype='object', name='sample', length=58581)

In [21]:
gene = [g.replace('.', '_') for g in gene]

In [22]:
pandasDF.columns = gene

In [23]:
start = time.time()
sparkDF=spark.createDataFrame(pandasDF.reset_index()) 
print("time:", time.time() - start)

time: 452.7069847583771


In [24]:
del pandasDF

In [25]:
gene = sparkDF.columns[1:]

In [26]:
gene

['CTD-2588J6_1',
 'RP11-433M22_1',
 'CTD-2588J6_2',
 'RP3-415N12_1',
 'RP11-16E12_2',
 'RP11-433M22_2',
 'PSMA2P3',
 'RP11-181G12_4',
 'RP11-181G12_2',
 'AC007272_3',
 'OR10V3P',
 'IGHV7-34-1',
 'CPHL1P',
 'CTD-2333K2_1',
 'AC016683_5',
 'RP11-654A16_3',
 'RP1-20N18_4',
 'TCOF1',
 'NSRP1',
 'RP5-1000K24_2',
 'RP11-285A1_1',
 'SPPL3',
 'AF001548_5',
 'AP000867_14',
 'OPA3',
 'OPA1',
 'RP1-68D18_2',
 'RP11-476K15_1',
 'RP1-278O22_1',
 'AC090587_5',
 'AC090587_4',
 'RP11-489D6_2',
 'RP11-666A8_7',
 'RP11-666A8_8',
 'AL359853_1',
 'RNU6-1209P',
 'RP11-190A12_9',
 'RNU4ATAC18P',
 'AC092652_1',
 'RNF219-AS1',
 'RP11-61E11_2',
 'RP11-61E11_1',
 'ITGA8',
 'ITGA9',
 'CTA-113A6_1',
 'AC108938_5',
 'RP11-146D12_2',
 'ITGA1',
 'ITGA2',
 'ITGA3',
 'ITGA4',
 'ITGA5',
 'ITGA6',
 'ITGA7',
 'TRHR',
 'AC010468_3',
 'AC010468_2',
 'AC010468_1',
 'RP11-428L21_2',
 'CA15P1',
 'CA15P2',
 'UFSP1',
 'UFSP2',
 'RP11-566K19_5',
 'RP11-566K19_6',
 'RP11-618L22_1',
 'RP11-566K19_8',
 'RP11-566K19_9',
 'C9orf40',


In [27]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols = gene,
    outputCol = 'genes')

df = assembler.transform(sparkDF).select("genes")

sparkDF.unpersist()
del sparkDF
gc.collect()

In [28]:
start = time.time()
pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
print("time:", time.time() - start)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 44514)
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_retu

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-061e853d6785>", line 2, in <module>
    pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
  File "/usr/local/spark/python/pyspark/ml/stat.py", line 165, in corr
    return _java2py(sc, javaCorrObj.corr(*args))
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling 

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39141)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializ

In [ ]:
print(str(pearsonCorr).replace('nan', 'NaN'))

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializ

In [ ]:
df = spark.createDataFrame(pearsonCorr,gene)

In [ ]:
import matplotlib.pyplot as plt

def plot_corr_matrix(correlations,attr,fig_no):
    fig=plt.figure(fig_no)
    ax=fig.add_subplot(111)
    ax.set_title("Correlation Matrix for Specified Attributes")
    ax.set_xticklabels(['']+attr)
    ax.set_yticklabels(['']+attr)
    cax=ax.matshow(correlations,vmax=1,vmin=-1)
    fig.colorbar(cax)
    plt.show()

plot_corr_matrix(df, gene, 234)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39141)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pyspark/serializ

# https://stackoverflow.com/questions/55546467/how-to-plot-correlation-heatmap-when-using-pysparkdatabricks

import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

columns = ['col1','col2','col3']

myGraph=spark.createDataFrame([(1.3,2.1,3.0),
                               (2.5,4.6,3.1),
                               (6.5,7.2,10.0)],
                              columns)
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=['col1','col2','col3'], 
                            outputCol=vector_col)
myGraph_vector = assembler.transform(myGraph).select(vector_col)
matrix = Correlation.corr(myGraph_vector, vector_col)

matrix = Correlation.corr(myGraph_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
print(corrmatrix)

df = spark.createDataFrame(corrmatrix,columns)
df.show()

def plot_corr_matrix(correlations,attr,fig_no):
    fig=plt.figure(fig_no)
    ax=fig.add_subplot(111)
    ax.set_title("Correlation Matrix for Specified Attributes")
    ax.set_xticklabels(['']+attr)
    ax.set_yticklabels(['']+attr)
    cax=ax.matshow(correlations,vmax=1,vmin=-1)
    fig.colorbar(cax)
    plt.show()

plot_corr_matrix(corrmatrix, columns, 234)

In [61]:
GTEx_breast_sample

['GTEX-T6MN-0726-SM-32PML',
 'GTEX-13NZB-2126-SM-5MR4Y',
 'GTEX-ZU9S-1926-SM-5NQBP',
 'GTEX-11DXW-0626-SM-5N9ER',
 'GTEX-UPK5-2326-SM-3P5Z8',
 'GTEX-VUSG-2226-SM-4KKZO',
 'GTEX-ZVTK-0326-SM-51MRR',
 'GTEX-14ABY-2126-SM-5QGQZ',
 'GTEX-R55F-1726-SM-2TF4R',
 'GTEX-U3ZH-1426-SM-4DXSR',
 'GTEX-13CF2-2026-SM-5K7VI',
 'GTEX-QEL4-2126-SM-447AE',
 'GTEX-11P7K-0726-SM-5EGKX',
 'GTEX-11EI6-0626-SM-5985T',
 'GTEX-11GS4-2126-SM-5A5KR',
 'GTEX-147F4-2826-SM-5NQBN',
 'GTEX-WYJK-1326-SM-3NB2T',
 'GTEX-145MO-0826-SM-5NQBL',
 'GTEX-Y111-2026-SM-4SOJA',
 'GTEX-13NZ9-1026-SM-5MR5K',
 'GTEX-QEG5-0726-SM-4R1JQ',
 'GTEX-111YS-1926-SM-5GICC',
 'GTEX-S33H-0326-SM-4AD6N',
 'GTEX-11PRG-0826-SM-5EQ6A',
 'GTEX-ZTX8-1226-SM-4YCE9',
 'GTEX-U412-1826-SM-4DXTJ',
 'GTEX-1192X-2326-SM-5987X',
 'GTEX-T2YK-2226-SM-32QPT',
 'GTEX-ZEX8-2226-SM-57WC6',
 'GTEX-139TU-0626-SM-5KM3X',
 'GTEX-SIU7-1626-SM-4BRUK',
 'GTEX-XMD1-0826-SM-4AT52',
 'GTEX-ZY6K-1626-SM-5GZWV',
 'GTEX-XBED-1626-SM-47JYN',
 'GTEX-11TT1-2126-SM-5GU5Y',
 'GTE

In [29]:
s = GTEx_breast_sample[0]
print(s)

GTEX-T6MN-0726-SM-32PML


In [76]:
GTEx_Breast_norm_count.printSchema()

root
 |-- sample: string (nullable = true)
 |-- GTEX-T6MN-0726-SM-32PML: double (nullable = true)
 |-- GTEX-13NZB-2126-SM-5MR4Y: double (nullable = true)
 |-- GTEX-ZU9S-1926-SM-5NQBP: double (nullable = true)
 |-- GTEX-11DXW-0626-SM-5N9ER: double (nullable = true)
 |-- GTEX-UPK5-2326-SM-3P5Z8: double (nullable = true)
 |-- GTEX-VUSG-2226-SM-4KKZO: double (nullable = true)
 |-- GTEX-ZVTK-0326-SM-51MRR: double (nullable = true)
 |-- GTEX-14ABY-2126-SM-5QGQZ: double (nullable = true)
 |-- GTEX-R55F-1726-SM-2TF4R: double (nullable = true)
 |-- GTEX-U3ZH-1426-SM-4DXSR: double (nullable = true)
 |-- GTEX-13CF2-2026-SM-5K7VI: double (nullable = true)
 |-- GTEX-QEL4-2126-SM-447AE: double (nullable = true)
 |-- GTEX-11P7K-0726-SM-5EGKX: double (nullable = true)
 |-- GTEX-11EI6-0626-SM-5985T: double (nullable = true)
 |-- GTEX-11GS4-2126-SM-5A5KR: double (nullable = true)
 |-- GTEX-147F4-2826-SM-5NQBN: double (nullable = true)
 |-- GTEX-WYJK-1326-SM-3NB2T: double (nullable = true)
 |-- GTEX-145M

In [64]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=GTEx_breast_sample, outputCols=GTEx_breast_sample)
model = imputer.fit(GTEx_Breast_norm_count)

In [65]:
model.transform(GTEx_Breast_norm_count).show()

+-------------+-----------------------+------------------------+-----------------------+------------------------+-----------------------+-----------------------+-----------------------+------------------------+-----------------------+-----------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+-----------------------+------------------------+-----------------------+------------------------+-----------------------+------------------------+-----------------------+-----------------------+------------------------+-----------------------+-----------------------+------------------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------------+-----------------------+------------------------+------------------------+-----------------------+-----------------------+--------

In [66]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols = GTEx_breast_sample,
    outputCol = 'genes')

In [67]:
output = assembler.transform(GTEx_Breast_norm_count)

In [77]:
df = output.select("genes")

In [78]:
from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation

In [82]:
start = time.time()
pearsonCorr = Correlation.corr(df, 'genes', 'pearson').collect()[0][0]
print("time:", time.time() - start)

time: 204.98473930358887


In [83]:
pearsonCorr

DenseMatrix(179, 179, [1.0, 0.9792, 0.9827, 0.9793, 0.9798, 0.9693, 0.9817, 0.9672, ..., 0.964, 0.9698, 0.9764, 0.9657, 0.9724, 0.9654, 0.969, 1.0], False)

In [84]:
print(str(pearsonCorr).replace('nan', 'NaN'))

DenseMatrix([[1.        , 0.97917421, 0.98267586, ..., 0.94388861, 0.95361579,
              0.96481694],
             [0.97917421, 1.        , 0.98364688, ..., 0.94814755, 0.95812364,
              0.96765248],
             [0.98267586, 0.98364688, 1.        , ..., 0.94923644, 0.95872443,
              0.96830039],
             ...,
             [0.94388861, 0.94814755, 0.94923644, ..., 1.        , 0.97886458,
              0.96543121],
             [0.95361579, 0.95812364, 0.95872443, ..., 0.97886458, 1.        ,
              0.9690076 ],
             [0.96481694, 0.96765248, 0.96830039, ..., 0.96543121, 0.9690076 ,
              1.        ]])


In [34]:
gene = GTEx_Breast_norm_count.select('sample').rdd.flatMap(lambda x: x).collect()

In [37]:
g = gene[0]
print(g)

CTD-2588J6.1


In [38]:
from pyspark.ml.functions import vector_to_array

In [ ]:
(GTEx_Breast_norm_count
 .withColumn(s, vector_to_array("vector")))
.select(["word"] + [col("xs")[i] for i in range(3)]))

Ensembl_gene_type = spark.read.options(header=True, inferSchema=True, delimiter=',').csv("../data/Ensembl_gene_name_type.csv")

genetype = Ensembl_gene_type.createOrReplaceTempView("genetype")

genelist = TcgaTargetGtex_RSEM_Hugo_norm_count.select("sample").rdd.flatMap(lambda x: x).collect()

genes = TcgaTargetGtex_RSEM_Hugo_norm_count.select("sample")

gene = genes.createOrReplaceTempView("gene")

Ensembl_TCGA_GETx_gene_type = spark.sql("SELECT Gene_name, Gene_type FROM gene INNER JOIN genetype ON gene.sample = genetype.gene_name")

Ensembl_TCGA_GETx_gene_type.write.format("csv").save("../data/Ensembl_TCGA_GETx_gene_type.csv")

In [25]:
#Ensembl_TCGA_GETx_gene_type.groupby("Gene_type").count()

Gene_type,count
protein_coding,21721
miRNA,2743
lincRNA,6589


In [13]:
type(GTEx_breast_sample)

list

In [14]:
test = ['sample']

In [15]:
type(test)

list

In [16]:
test.extend(GTEx_breast_sample)

In [17]:
GTEx_Breast_norm_count = TcgaTargetGtex_RSEM_Hugo_norm_count.select(test)

In [18]:
pandasDF = GTEx_Breast_norm_count.toPandas().set_index('sample').transpose()

In [19]:
pandasDF.head()

sample,CTD-2588J6.1,RP11-433M22.1,CTD-2588J6.2,RP3-415N12.1,RP11-16E12.2,RP11-433M22.2,PSMA2P3,RP11-181G12.4,RP11-181G12.2,AC007272.3,...,RP11-390F4.3,GNGT2,GNGT1,SELT,DIAPH2-AS1,SELV,NFIX,SELP,AC002386.1,RP11-526P5.2
GTEX-T6MN-0726-SM-32PML,1.4482,0.0,0.0,2.8177,3.5074,0.8986,0.0000,0.8986,6.8026,0.0000,...,6.4923,3.6131,0.0000,11.0095,1.4482,1.4482,13.2609,9.8067,0.0,0.0
GTEX-13NZB-2126-SM-5MR4Y,0.0000,0.0,0.0,0.0000,1.0154,0.0000,0.0000,0.0000,6.6302,0.0000,...,5.8881,4.7844,0.0000,11.0757,1.0154,3.1972,12.6347,9.8917,0.0,0.0
GTEX-ZU9S-1926-SM-5NQBP,0.0000,0.0,0.0,1.4965,1.9000,0.0000,0.0000,0.0000,6.7367,0.0000,...,6.3745,4.5215,0.0000,11.4081,2.5108,3.9609,13.2720,8.6533,0.0,0.0
GTEX-11DXW-0626-SM-5N9ER,0.0000,0.0,0.0,1.0942,2.4699,0.0000,0.0000,1.0942,6.5205,0.0000,...,5.7641,4.7636,0.0000,10.8074,1.0942,1.7093,12.8925,11.0399,0.0,0.0
GTEX-UPK5-2326-SM-3P5Z8,1.2686,0.0,0.0,1.9330,2.7305,0.0000,1.2686,0.0000,6.5973,1.8979,...,5.6328,5.2937,0.7694,10.9437,2.5686,1.2686,12.6923,11.0987,0.0,0.0


In [20]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")

In [21]:
sparkDF=spark.createDataFrame(pandasDF.reset_index()) 

In [22]:
sparkDF.columns[0:10]

['index',
 'CTD-2588J6.1',
 'RP11-433M22.1',
 'CTD-2588J6.2',
 'RP3-415N12.1',
 'RP11-16E12.2',
 'RP11-433M22.2',
 'PSMA2P3',
 'RP11-181G12.4',
 'RP11-181G12.2']

In [25]:
sparkDF.head(1)

Py4JJavaError: An error occurred while calling o325.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at java.base/java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:120)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:95)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:156)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1848)
	at java.base/java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1536)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1510)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputS

In [23]:
import time

start = time.time()
sparkDF.corr('CTD-2588J6.1', 'RP11-433M22.1')
print("time:", time.time() - start)

AnalysisException: Cannot resolve column name "CTD-2588J6.1" among (index, CTD-2588J6.1, RP11-433M22.1, CTD-2588J6.2, RP3-415N12.1, RP11-16E12.2, RP11-433M22.2, PSMA2P3, RP11-181G12.4, RP11-181G12.2, AC007272.3, OR10V3P, IGHV7-34-1, CPHL1P, CTD-2333K2.1, AC016683.5, RP11-654A16.3, RP1-20N18.4, TCOF1, NSRP1, RP5-1000K24.2, RP11-285A1.1, SPPL3, AF001548.5, AP000867.14, OPA3, OPA1, RP1-68D18.2, RP11-476K15.1, RP1-278O22.1, AC090587.5, AC090587.4, RP11-489D6.2, RP11-666A8.7, RP11-666A8.8, AL359853.1, RNU6-1209P, RP11-190A12.9, RNU4ATAC18P, AC092652.1, RNF219-AS1, RP11-61E11.2, RP11-61E11.1, ITGA8, ITGA9, CTA-113A6.1, AC108938.5, RP11-146D12.2, ITGA1, ITGA2, ITGA3, ITGA4, ITGA5, ITGA6, ITGA7, TRHR, AC010468.3, AC010468.2, AC010468.1, RP11-428L21.2, CA15P1, CA15P2, UFSP1, UFSP2, RP11-566K19.5, RP11-566K19.6, RP11-618L22.1, RP11-566K19.8, RP11-566K19.9, C9orf40, RP11-379H18.1, NUDT19P5, RP11-1094M14.7, TEX28P2, TEX28P1, AC141002.1, AC003688.1, FAM212B, FAM212A, CHST9, CHST8, CHST1, RP11-155G1

In [ ]:
sparkDF.printSchema()

In [ ]:
sparkDF.show()

https://stackoverflow.com/questions/60430645/get-a-correlation-matrix-in-pyspark-with-a-large-dataset

In [ ]:
df = GTEx_Breast_norm_count.select(GTEx_breast_sample)

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=df.columns, outputCol=vector_col)
df_vector = assembler.transform(df).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)

In [83]:
matrix

pearson(corr_features)
1.0 ...


In [96]:
corr = matrix.collect()[0]["pearson({})".format(vector_col)].values